In [38]:
import numpy as np
def solve_pH(Sh_ini, StV, Keq, chrM):    
    w = 1
    a, b = 1e-14, 1
    spcM = np.zeros(np.shape(chrM))
    G = []
    for Sh in [a,b]:
        Denm = Sh**3 + Keq[:, 0] * Sh**2 + Keq[:, 1] * Keq[:, 0] * Sh + Keq[:, 2]* Keq[:, 1]* Keq[:, 0]
        spcM[:, 0] = (StV * Sh**3) / Denm
        spcM[:, 1] = (StV * Sh**2 * Keq[:, 0]) / Denm
        spcM[:, 2] = (StV * Sh * Keq[:, 0]* Keq[:, 1]) / Denm
        spcM[:, 3] = (StV * Keq[:, 0] * Keq[:, 1]* Keq[:, 2]) / Denm
        G.append(Sh + sum(sum(spcM * chrM)) - 10**(-14)/Sh)
    fa, fb = G[0], G[1]
    if fa*fb > 0:
        print('Wrong input')
        Sh, spcM, ipH = 0, 0, 0
        return Sh, spcM, ipH
    # Newton-Raphson method.-
    Sh = Sh_ini
    #Counter of convergences
    ipH = 0; Tol = 5.e-15; maxIter = 100
    dspcM = spcM
    while ipH < maxIter:
        Denm = Sh**3 + Keq[:, 0] * Sh**2 + Keq[:, 1] * Keq[:, 0] * Sh + Keq[:, 2]* Keq[:, 1]* Keq[:, 0]
        spcM[:, 0] = (StV * Sh**3) / Denm
        spcM[:, 1] = (StV * Sh**2 * Keq[:, 0]) / Denm
        spcM[:, 2] = (StV * Sh * Keq[:, 0]* Keq[:, 1]) / Denm
        spcM[:, 3] = (StV * Keq[:, 0] * Keq[:, 1]* Keq[:, 2]) / Denm   
      
        # Evaluation of the charge balance for the current Sh value, F(Sh)
        F = Sh + sum(sum(spcM * chrM)) - 10**(-14)/Sh

        # Calculation of all derivated functions
        dDenm = Denm**2;
        aux = 3 * Sh**2 + 2 * Sh * Keq[:, 0] + Keq[:, 0]* Keq[:, 1]

        dspcM[:, 0] = (3 * Sh**2 *StV) / Denm - (StV * Sh**3 * aux) / dDenm
        dspcM[:, 1] = (2 * Sh * Keq[:, 0] * StV) / Denm - ((Keq[:, 0]* StV * Sh**2) * aux) / dDenm
        dspcM[:, 2] = (Keq[:, 0] * Keq[:, 1] * StV) / Denm - ((Keq[:, 0] * Keq[:, 1] * StV * Sh) * aux) / dDenm
        dspcM[:, 3] = -(Keq[:, 0] * Keq[:, 1] * Keq[:, 2] * StV * aux) / dDenm

        # Evaluation of the charge balance for the current Sh value, dF(Sh)
        dF = 1 + sum(sum(dspcM * chrM)) + 10**(-14)/Sh**2
        #Error
        err = F / dF
        #Newton-Raphson algorithm
        Sh = Sh - err

        if (abs(err) < 1e-14) and (abs(F) < Tol):
            # Checking if the input gives a valid pH 
            if (Sh > 1e-14) and (Sh < 1):
                break
            else:
                ipH = 0; err1, err2 = 1, 1
                while (err1 > Tol): #and (err2 > 1e-14):
                    Sh = (fb * a - fa * b) / (fb - fa)
                    Denm = Sh**3 + Keq[:, 0] * Sh**2 + Keq[:, 1] * Keq[:, 0] * Sh + Keq[:, 2]* Keq[:, 1]* Keq[:, 0]
                    spcM[:, 0] = (StV * Sh**3) / Denm
                    spcM[:, 1] = (StV * Sh**2 * Keq[:, 0]) / Denm
                    spcM[:, 2] = (StV * Sh * Keq[:, 0]* Keq[:, 1]) / Denm
                    spcM[:, 3] = (StV * Keq[:, 0] * Keq[:, 1]* Keq[:, 2]) / Denm     
                    fc = Sh + sum(sum(spcM * chrM)) - 10**(-14)/Sh
                    if fa * fc > 0:
                        a, fa = Sh, fc
                    elif fb * fc > 0: # To avoid problems when fc == 0
                        b, fb = Sh, fc

                    err1 = abs(fc)
                    #err2 = abs(Sh - (fb * a - fa * b) / (fb - fa))
                    ipH += 1
        ipH += 1
            

    return Sh, spcM, ipH


In [39]:
#CASE 1
#order: inorganic carbon - phosphate - acetic ac - ammonium - cations - anions
Keq = np.array([[ 10**(-6.35), 10**(-10.33),  0],[ 10**(-2.14), 10**(-7.2), 10**(-12.37)],[10**(-4.75), 0,0],[10**(-9.3),0,0],[0,0,0],[0,0,0]])
chrM = np.array([[0,-1,-2, 0],[0,-1,-2, -3], [ 0, -1, 0, 0],[ 1, 0, 0, 0], [ 1, 0, 0, 0], [-1, 0, 0, 0]])
Sh_ini = 10**(-7)
ic, phosphate, acetic, nh4 = 5*10**(-3), 1*10**(-6), 3*10**(-3), 2*10**(-3)
cations = 3*phosphate + ic + 2*10**(-3)
anions = 2*10**(-3)
StV =np.array([ic, phosphate, acetic, nh4, cations, anions])
[Sh, spcM, ipH] =solve_pH(Sh_ini, StV, Keq, chrM)
pH = -np.log10(Sh)
pH, ipH, spcM

(6.957013911271398,
 3,
 array([[ 7.19772662e+03, -7.17930611e+03, -1.84205064e+01,
         -0.00000000e+00],
        [ 1.19780342e-04,  2.09595943e+00, -2.09605839e+00,
         -2.08256585e-05],
        [ 1.66626971e+02, -1.66626971e+02,  0.00000000e+00,
         -0.00000000e+00],
        [ 8.14936421e+01, -8.14936421e+01,  0.00000000e+00,
         -0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00],
        [-7.27595761e-12,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00]]))

In [40]:
#CASE 2
#order: inorganic carbon - phosphate - acetic ac - ammonium - cations - anions
Keq = np.array([[ 10**(-6.35), 10**(-10.33),  0],[ 10**(-2.14), 10**(-7.2), 10**(-12.37)],[10**(-4.75), 0,0],[10**(-9.3),0,0],[0,0,0],[0,0,0]])
chrM = np.array([[0,-1,-2, 0],[0,-1,-2, -3], [ 0, -1, 0, 0],[ 1, 0, 0, 0], [ 1, 0, 0, 0], [-1, 0, 0, 0]])
Sh_ini = 10**(-7)
ic, phosphate, acetic, nh4 = 15*10**(-3), 1*10**(-4), 21*10**(-3), 2*10**(-3)
cations = 3*phosphate + ic + 22*10**(-3)
anions = 2*10**(-3)
StV =np.array([ic, phosphate, acetic, nh4, cations, anions])
[Sh, spcM, ipH] =solve_pH(Sh_ini, StV, Keq, chrM)
pH = -np.log10(Sh)
pH, ipH, spcM

(8.913504653092218,
 29785,
 array([[3.93650884e-05, 1.44084042e-02, 5.52230688e-04, 0.00000000e+00],
        [3.19537614e-13, 1.89682228e-06, 9.80688978e-05, 3.42796159e-08],
        [1.44106921e-06, 2.09985589e-02, 0.00000000e+00, 0.00000000e+00],
        [1.41775493e-03, 5.82245069e-04, 0.00000000e+00, 0.00000000e+00],
        [3.73000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.00000000e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]))